In [ ]:
!pip install openai

from openai import OpenAI

from google.colab import userdata


# OpenAI API 클라이언트 생성
client = OpenAI(api_key=userdata.get('api_key'))

In [7]:

import os, json, textwrap


def safe_chat(**kwargs):
    """
    Chat Completions 래퍼: 오류를 잡아 깔끔히 출력
    """
    try:
        resp = client.chat.completions.create(**kwargs)
        return resp.choices[0].message
    except Exception as e:
        print(" 호출 실패:", repr(e))
        return None


In [8]:
STYLE_GUIDE = textwrap.dedent("""
[역할] 너는 비전공자에게도 이해되게 쓰는 한국어 기술 작가다.
[목표] 아래 주제를 뉴스레터 형식으로 6~8문장, 최대 1200자.
[형식]
- 제목 1줄 (이모지 1개), 본문 3~5문단, 마지막에  요약 2줄
- 문단은 2~3문장씩, 불필요한 수식/영어 줄이기
[금지어] '혁신적이다', '게임체인저', '만약 ~라면' 금지
[검증] 사실 단정 대신 '주로', '일반적으로' 등 완곡한 표현
""").strip()

topic = "벡터 임베딩이 검색/추천에서 쓰이는 이유"
msg = safe_chat(
    model="gpt-4o-mini",
    messages=[
        {"role":"system","content":STYLE_GUIDE},
        {"role":"user","content":f"주제: {topic}"}
    ],
    temperature=0.3,
    seed=7
)
print(msg.content if msg else "실패")


### 벡터 임베딩이 검색과 추천에서 중요한 이유 🔍

벡터 임베딩은 데이터를 숫자로 표현하는 방법으로, 주로 검색과 추천 시스템에서 활용됩니다. 이 기술은 단어, 이미지, 사용자 행동 등을 벡터라는 형태로 변환하여, 서로의 유사성을 쉽게 비교할 수 있도록 도와줍니다. 예를 들어, 비슷한 취향을 가진 사용자나 유사한 제품을 찾는 데 유용합니다.

검색 엔진에서는 사용자가 입력한 키워드를 벡터로 변환하여, 데이터베이스에 있는 문서와의 유사도를 계산합니다. 이를 통해 사용자가 원하는 정보를 더 정확하게 찾을 수 있습니다. 또한, 추천 시스템에서는 사용자의 과거 행동을 분석하여, 그와 비슷한 다른 사용자들이 선호하는 아이템을 추천합니다.

이러한 벡터 임베딩 기술은 대량의 데이터를 처리하는 데 효율적이며, 사용자의 경험을 향상시키는 데 기여합니다. 특히, 자연어 처리와 이미지 인식 분야에서도 널리 사용되고 있습니다. 데이터의 복잡성을 줄이고, 더 나은 결과를 제공하는 데 중요한 역할을 합니다.

요약하자면, 벡터 임베딩은 검색과 추천 시스템에서 데이터의 유사성을 비교하는 데 필수적인 기술입니다. 이를 통해 사용자에게 더 나은 정보를 제공하고, 개인화된 경험을 만들어갑니다.


환각억제

In [9]:
CONTEXT = textwrap.dedent("""
<<자료>>
- 임베딩은 텍스트/아이템을 고정 길이 벡터로 바꾸는 표현 학습 기법이다.
- 코사인 유사도 등으로 의미적 근접도를 계산한다.
- 추천에서는 '유사 사용자/아이템'을 근거로 후보를 확장한다.
<</자료>>
""").strip()

prompt = f"""
규칙:
1) 반드시 <<자료>>만 근거로 사용. 자료 밖 내용 추측 금지.
2) 답 마지막에 [근거] 섹션으로 사용한 문장 1~2개만 인용.
3) 만약 질문이 자료에 없으면 '모름'이라고만 답.

질문: '임베딩이 추천 시스템에서 주는 이점 2가지?'

{CONTEXT}
"""

msg = safe_chat(
    model="gpt-4o-mini",
    messages=[
        {"role":"system","content":"간결하고 정확하게 답하라."},
        {"role":"user","content":prompt}
    ],
    temperature=0.0
)
print(msg.content if msg else "실패")


임베딩이 추천 시스템에서 주는 이점 2가지는 다음과 같습니다. 첫째, 텍스트나 아이템을 고정 길이 벡터로 변환하여 효율적인 데이터 처리가 가능하다. 둘째, 코사인 유사도 등을 통해 의미적 근접도를 계산하여 유사 사용자나 아이템을 기반으로 후보를 확장할 수 있다. 

[근거] "임베딩은 텍스트/아이템을 고정 길이 벡터로 바꾸는 표현 학습 기법이다." / "추천에서는 '유사 사용자/아이템'을 근거로 후보를 확장한다."


Few-shot : 단 몇 개의 예시(examples)를 프롬프트(지시문)에 포함하여 제공하는 방법(In-context Learning)

| 개념              | 설명                                                                 | 예시 (감정 분석) |
|-------------------|----------------------------------------------------------------------|------------------|
| **Zero-shot (제로샷)** | 예시 없이, 오직 지시만으로 작업을 요청. 모델의 기존 지식에 전적으로 의존. | "이 문장은 긍정이야 부정이야?" |
| **One-shot (원샷)**    | 단 하나의 예시를 제공. 원하는 출력 형식을 명확히 할 때 유용.            | 입력: "배송 빨라요." -> 긍정<br>입력: "별로네요." -> ? |
| **Few-shot (퓨샷)**    | 몇 개(2~5개)의 예시를 제공. 더 복잡한 패턴이나 뉘앙스를 학습시키기에 효과적. | (위의 구체적인 예시와 동일) |
| **Fine-tuning (파인튜닝)** | 수백~수천 개 이상의 많은 데이터를 사용해 모델의 가중치(weights)를 직접 업데이트. 모델을 특정 작업에 깊게 전문화시키는 과정. | 감정분석 데이터셋으로 모델을 재훈련 |



few-shot 으로 "role":"assistant" 이용

In [10]:
LABEL_GUIDE = textwrap.dedent("""
[라벨 정의]
- 긍정: 성능/품질/경험에 대한 우호적 평가
- 중립: 사실 전달 또는 복합 감정(장단 공존)
- 부정: 문제/불만/불편 호소
[출력] JSON {"label": "긍정|중립|부정", "rationale": "한글 1~2문장"}
""").strip()

shots = [
  {"role":"user","content":"문장: '검색 품질이 많이 좋아졌다!'"},
  {"role":"assistant","content":'{"label":"긍정","rationale":"품질 향상에 대한 우호적 표현."}'},
  {"role":"user","content":"문장: '속도는 괜찮지만 정확도는 흔들린다.'"},
  {"role":"assistant","content":'{"label":"중립","rationale":"장단이 동시에 언급됨."}'},
]

query = "문장: '결과가 자꾸 틀려서 답답하다.'"

msg = safe_chat(
    model="gpt-4o-mini",
    messages=[{"role":"system","content":LABEL_GUIDE}] + shots + [{"role":"user","content":query}],
    response_format={"type":"json_object"},
    temperature=0.0
)
print(json.dumps(json.loads(msg.content), ensure_ascii=False, indent=2) if msg else "실패")


{
  "label": "부정",
  "rationale": "결과의 부정확성에 대한 불만 표현."
}


In [ ]:

STYLE_GUIDE = textwrap.dedent("""
[역할] 너는 취업 준비생을 돕는 자기소개서 작성 가이드다.
[목표] 아래 주제를 바탕으로 자기소개서 문단 예시를 5~7문장, 800자 이내로 작성한다.
[형식]
- 1문단 구성, 문장은 2~3문장씩 연결
- 지원 동기, 경험, 강점이 드러나도록
- 불필요한 수식어나 과장 표현 금지
[금지어] '열정적이다', '최고', '무조건' 등의 과도한 표현 금지
[검증] 사실은 구체적인 경험 중심으로, 장점은 완곡하게 표현
""").strip()

topic = "협업 경험을 강조한 자기소개서"

msg = safe_chat(
    model="gpt-4o-mini",
    messages=[
        {"role":"system","content": STYLE_GUIDE},
        {"role":"user","content": f"주제: {topic}"}
    ],
    temperature=0.8,
    seed=7
)
print(msg.content if msg else "실패")